In [29]:
import numpy as np 
import pandas as pd 
import os
for dirname, _, filenames in os.walk('D:/WORK-USA/flower'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

D:/WORK-USA/flower\test\00-192x192-462.tfrec
D:/WORK-USA/flower\test\01-192x192-462.tfrec
D:/WORK-USA/flower\test\02-192x192-462.tfrec
D:/WORK-USA/flower\test\03-192x192-462.tfrec
D:/WORK-USA/flower\test\04-192x192-462.tfrec
D:/WORK-USA/flower\test\05-192x192-462.tfrec
D:/WORK-USA/flower\test\06-192x192-462.tfrec
D:/WORK-USA/flower\test\07-192x192-462.tfrec
D:/WORK-USA/flower\test\08-192x192-462.tfrec
D:/WORK-USA/flower\test\09-192x192-462.tfrec
D:/WORK-USA/flower\test\10-192x192-462.tfrec
D:/WORK-USA/flower\test\11-192x192-462.tfrec
D:/WORK-USA/flower\test\12-192x192-462.tfrec
D:/WORK-USA/flower\test\13-192x192-462.tfrec
D:/WORK-USA/flower\test\14-192x192-462.tfrec
D:/WORK-USA/flower\test\15-192x192-452.tfrec
D:/WORK-USA/flower\train\00-192x192-798.tfrec
D:/WORK-USA/flower\train\01-192x192-798.tfrec
D:/WORK-USA/flower\train\02-192x192-798.tfrec
D:/WORK-USA/flower\train\03-192x192-798.tfrec
D:/WORK-USA/flower\train\04-192x192-798.tfrec
D:/WORK-USA/flower\train\05-192x192-798.tfrec
D:/W

In [31]:
import math, re, os
import tensorflow as tf
import numpy as np
from matplotlib import pyplot as plt
from sklearn.metrics import f1_score, precision_score, recall_score, confusion_matrix, accuracy_score
from tensorflow.keras import backend as K
print("Tensorflow version " + tf.__version__)
AUTO = tf.data.experimental.AUTOTUNE

Tensorflow version 2.14.0


In [33]:
try:
    tpu = tf.distribute.cluster_resolver.TPUClusterResolver()  
    print('Running on TPU ', tpu.master())
except ValueError:
    tpu = None

if tpu:
    tf.config.experimental_connect_to_cluster(tpu)
    tf.tpu.experimental.initialize_tpu_system(tpu)
    strategy = tf.distribute.experimental.TPUStrategy(tpu)
else:
    strategy = tf.distribute.get_strategy() 

print("REPLICAS: ", strategy.num_replicas_in_sync)

REPLICAS:  1


In [34]:
IMAGE_SIZE = [192, 192]
EPOCHS = 20
BATCH_SIZE = 16 * strategy.num_replicas_in_sync

In [35]:
GCS_PATH="D:/WORK-USA/flower"
TRAINING_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/train/*.tfrec')
VALIDATION_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/val/*.tfrec')
TEST_FILENAMES = tf.io.gfile.glob(GCS_PATH + '/test/*.tfrec')

In [45]:
print(TRAINING_FILENAMES)
print(VALIDATION_FILENAMES)
print(TEST_FILENAMES)

['D:\\WORK-USA\\flower\\train\\00-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\01-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\02-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\03-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\04-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\05-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\06-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\07-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\08-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\09-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\10-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\11-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\12-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\13-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\14-192x192-798.tfrec', 'D:\\WORK-USA\\flower\\train\\15-192x192-783.tfrec']
['D:\\WORK-USA\\flower\\val\\00-192x192-232.tfrec', 'D:\\WORK-USA\\flower\\val\\01-192x192-232.tfrec', 'D:\\WORK-USA\\flower\\val\\02-192x192-232.tfrec

In [22]:
train_dataset = create_dataset(train_files, batch_size, shuffle_buffer_size)
val_dataset = create_dataset(val_files, batch_size, shuffle_buffer_size)
test_dataset = create_dataset(test_files, batch_size, shuffle_buffer_size)

In [46]:
CLASSES = ['pink primrose',    'hard-leaved pocket orchid', 'canterbury bells', 'sweet pea',     'wild geranium',     'tiger lily',           'moon orchid',              'bird of paradise', 'monkshood',        'globe thistle',         
           'snapdragon',       "colt's foot",               'king protea',      'spear thistle', 'yellow iris',       'globe-flower',         'purple coneflower',        'peruvian lily',    'balloon flower',   'giant white arum lily', 
           'fire lily',        'pincushion flower',         'fritillary',       'red ginger',    'grape hyacinth',    'corn poppy',           'prince of wales feathers', 'stemless gentian', 'artichoke',        'sweet william',         
           'carnation',        'garden phlox',              'love in the mist', 'cosmos',        'alpine sea holly',  'ruby-lipped cattleya', 'cape flower',              'great masterwort', 'siam tulip',       'lenten rose',           
           'barberton daisy',  'daffodil',                  'sword lily',       'poinsettia',    'bolero deep blue',  'wallflower',           'marigold',                 'buttercup',        'daisy',            'common dandelion',      
           'petunia',          'wild pansy',                'primula',          'sunflower',     'lilac hibiscus',    'bishop of llandaff',   'gaura',                    'geranium',         'orange dahlia',    'pink-yellow dahlia',    
           'cautleya spicata', 'japanese anemone',          'black-eyed susan', 'silverbush',    'californian poppy', 'osteospermum',         'spring crocus',            'iris',             'windflower',       'tree poppy',            
           'gazania',          'azalea',                    'water lily',       'rose',          'thorn apple',       'morning glory',        'passion flower',           'lotus',            'toad lily',        'anthurium',             
           'frangipani',       'clematis',                  'hibiscus',         'columbine',     'desert-rose',       'tree mallow',          'magnolia',                 'cyclamen ',        'watercress',       'canna lily',            
           'hippeastrum ',     'bee balm',                  'pink quill',       'foxglove',      'bougainvillea',     'camellia',             'mallow',                   'mexican petunia',  'bromelia',         'blanket flower',        
           'trumpet creeper',  'blackberry lily',           'common tulip',     'wild rose']                           